## import library

In [183]:
# %pip install llama-index-vector-stores-qdrant
# %pip install --upgrade llama-index
# %pip install --upgrade fastembed
# %pip install --upgrade llama-index-storage-chat-store-redis
# %pip install --upgrade llama-index-core
# %pip install --upgrade redis
# %pip install --upgrade llama-index-storage-kvstore-redis
# %pip install "numpy<2.0"
# %pip install --upgrade pandas pyarrow llama-index
# %pip install "pybind11>=2.12"
# %pip install --upgrade llama-index-retrievers-bm25

In [184]:
import os
from dotenv import load_dotenv

# Tải các biến môi trường từ file .env
load_dotenv()

# Lấy giá trị khóa API từ biến môi trường
openai_api_key = os.getenv("OPENAI_API_KEY")

In [185]:
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# Initialize the LLM
llm = OpenAI(
    model="gpt-4o-mini"
)

# Initialize the LLM embeddings
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
# Global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.context_window = 128_000
Settings.num_output = 1_000_000

## read and parse

In [186]:
import os
import tempfile
import pdfplumber
from typing import List
from llama_index.core import SimpleDirectoryReader

def process_single_pdf(pdf_path: str) -> List:
    """
    Load and parse a specific PDF file using SimpleDirectoryReader.
    Args:
        pdf_path (str): Path to the PDF file to process.

    Returns:
        List: Documents from SimpleDirectoryReader.
    """
    # Check file existence
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"File not found: {pdf_path}")

    # Check file extension
    if not pdf_path.lower().endswith('.pdf'):
        raise ValueError(f"File {pdf_path} is not a PDF file")

    # Create temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        try:
            pdf_filename = os.path.basename(pdf_path)
            temp_text_file = os.path.join(temp_dir, f'{os.path.splitext(pdf_filename)[0]}.txt')

            # Load PDF and save to temporary text file
            with pdfplumber.open(pdf_path) as pdf:
                text_parts = []  # Use a list to gather text parts
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:  # Only add if there's text
                        text_parts.append(page_text)

            # Write all text at once
            with open(temp_text_file, 'w', encoding='utf-8') as f:
                f.write(''.join(text_parts))

            print(f"File processed: {pdf_filename}")

            # Use SimpleDirectoryReader to read text files
            documents = SimpleDirectoryReader(temp_dir).load_data()

        except pdfplumber.PDFException as e:
            raise Exception(f"PDF processing error for file {pdf_filename}: {str(e)}")
        except Exception as e:
            raise Exception(f"Error processing file {pdf_filename}: {str(e)}")

    return documents

input_dir = "TULD02.pdf"
documents = process_single_pdf(input_dir)

print(documents)

File processed: TULD02.pdf
[Document(id_='04159a3e-8221-4148-af44-e6569352aabb', embedding=None, metadata={'file_path': 'C:\\Users\\LUNE\\AppData\\Local\\Temp\\tmphkfz7qvh\\TULD02.txt', 'file_name': 'TULD02.txt', 'file_type': 'text/plain', 'file_size': 16468, 'creation_date': '2024-12-02', 'last_modified_date': '2024-12-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\r\nĐộc lập - Tự do - Hạnh phúc\r\nTHỎA ƯỚC\r\nLAO ĐỘNG TẬP THỂ\r\nCăn cứ vào Bộ Luật lao động số 45/2019/QH14 ngày 20/11/2019;\r\nĐể đảm bảo quyền lợi và nghĩa vụ hợp pháp của mỗi bên trong quan hệ\r\nlao động, chúng tôi gồm có:\r\n1. Người sử dụng lao động:\r\nÔng: PHẠM NGỌC THUẬN - Tổng Gi

# define qdrant

In [187]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams

# Kết nối tới Qdrant (đang chạy cục bộ hoặc cloud)
qdrant_client = QdrantClient(url="http://localhost:6333")
collection_name = "documents_collection"

# Kiểm tra kết nối và danh sách các collections hiện có
collections = qdrant_client.get_collections()
print("Current collections:", collections)

# Kiểm tra xem collection đã tồn tại hay chưa
if collection_name not in [collection.name for collection in collections.collections]:
    # Tạo collection nếu chưa tồn tại
    vector_params = VectorParams(size=1536, distance="Cosine")  # Kích thước vector và khoảng cách cosine
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=vector_params,
        on_disk = True
    )
    print(f"Collection '{collection_name}' đã được tạo.")
else:
    print(f"Collection '{collection_name}' đã tồn tại.")

Current collections: collections=[CollectionDescription(name='documents_collection')]
Collection 'documents_collection' đã tồn tại.


In [188]:
import json
def save_to_qdrant(nodes):
    
    for node in nodes:
        text = node.text

        # Get metadata to save as payload dict
        metadata = node.metadata
        payload = dict(metadata)
        
        # Create embdding from text
        embedding = embed_model._get_text_embedding(text)
    

        try:
            qdrant_client.upsert(
                collection_name="documents_collection",
                points=[
                    PointStruct(
                        id=node.id_,
                        vector=embedding,
                        payload=payload
                        )
                    ]
                )
            print("Dữ liệu đã được lưu vào Qdrant!")
        except Exception as e:
            print(f"Error saving data to Qdrant: {e}")

In [189]:
from qdrant_client import QdrantClient

# Kiểm tra xem collection có tồn tại không
try:
    # Lấy thông tin về collection
    collection_info = qdrant_client.get_collection(collection_name)
    print(f"Collection '{collection_name}' đã được kết nối thành công.")
except Exception as e:
    print(f"Lỗi khi kết nối với collection: {str(e)}")


Collection 'documents_collection' đã được kết nối thành công.


In [190]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext

# Khởi tạo QdrantVectorStore với tên collection được cập nhật
vector_store = QdrantVectorStore(
    client=qdrant_client, 
    collection_name="documents_collection"  # Đổi tên collection tại đây
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
print(storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x000001FD5B561DF0>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x000001FD5BF595B0>, vector_stores={'default': QdrantVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=False, collection_name='documents_collection', url=None, api_key=None, batch_size=64, parallel=1, max_retries=3, client_kwargs={}, enable_hybrid=False, index_doc_id=True, fastembed_sparse_model=None, text_key='text'), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x000001FD5BF5B920>, property_graph_store=None)


# indexing & chunking & pipeline

In [191]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.extractors import QuestionsAnsweredExtractor, KeywordExtractor
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core import Settings
from qdrant_client import QdrantClient
import numpy as np
from datetime import datetime
import nest_asyncio

nest_asyncio.apply()

# Tạo vector_store sử dụng Qdrant
from llama_index.vector_stores.qdrant import QdrantVectorStore
vector_store = QdrantVectorStore(client=qdrant_client, collection_name="documents_collection")

# Cấu hình các extractor và node parser
extractors = [
    QuestionsAnsweredExtractor(llm=Settings.llm, questions=1),
    KeywordExtractor(llm=Settings.llm, keywords=5),
]

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=Settings.embed_model
)

# Các transformations
transformations = [splitter] + extractors

# Khởi tạo ingestion pipeline
pipeline = IngestionPipeline(
    transformations=transformations,
    vector_store=vector_store
)

# Chạy pipeline để xử lý documents
nodes = pipeline.run(documents=documents, show_progress=True, batch_size=64)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/122 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:01<00:00,  3.54it/s]


In [192]:
# Beutifulize print nodes
import json
print("Nodes:")
for node in nodes:
    print(node)

    # metadata
    # print("Metadata:")
    print(json.dumps(node.metadata, indent=2))
    break

Nodes:
Node ID: e2ff35f1-26f5-487e-a106-65e9a81f6b2b
Text: CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM  Độc lập - Tự do - Hạnh phúc
THỎA ƯỚC  LAO ĐỘNG TẬP THỂ  Căn cứ vào Bộ Luật lao động số
45/2019/QH14 ngày 20/11/2019;  Để đảm bảo quyền lợi và nghĩa vụ hợp
pháp của mỗi bên trong quan hệ  lao động, chúng tôi gồm có:  1. Người
sử dụng lao động:  Ông: PHẠM NGỌC THUẬN - Tổng Giám đốc Tổng Công ty.
{
  "file_path": "C:\\Users\\LUNE\\AppData\\Local\\Temp\\tmphkfz7qvh\\TULD02.txt",
  "file_name": "TULD02.txt",
  "file_type": "text/plain",
  "file_size": 16468,
  "creation_date": "2024-12-02",
  "last_modified_date": "2024-12-02",
  "questions_this_excerpt_can_answer": "What is the name and title of the employer mentioned in the labor collective agreement?",
  "excerpt_keywords": "Keywords: labor collective agreement, employer, Ph\u1ea1m Ng\u1ecdc Thu\u1eadn, T\u1ed5ng Gi\u00e1m \u0111\u1ed1c, Vietnam"
}


In [193]:
from qdrant_client.models import PointStruct

In [194]:
import json
def save_to_qdrant(nodes):
    
    for node in nodes:
        text = node.text

        # Get metadata to save as payload dict
        metadata = node.metadata
        payload = dict(metadata)
        
        # Create embdding from text
        embedding = embed_model._get_text_embedding(text)
    

        try:
            qdrant_client.upsert(
                collection_name="documents_collection",
                points=[
                    PointStruct(
                        id=node.id_,
                        vector=embedding,
                        payload=payload
                        )
                    ]
                )
            print("Dữ liệu đã được lưu vào Qdrant!")
        except Exception as e:
            print(f"Error saving data to Qdrant: {e}")
# Lưu các nodes vào Qdrant
qdrant_client.recreate_collection(
    collection_name="documents_collection",
    vectors_config=VectorParams(size=1536, distance="Cosine"),
)

C:\Users\LUNE\AppData\Local\Temp\ipykernel_8480\633579027.py:30: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [195]:
save_to_qdrant(nodes)

Dữ liệu đã được lưu vào Qdrant!
Dữ liệu đã được lưu vào Qdrant!
Dữ liệu đã được lưu vào Qdrant!
Dữ liệu đã được lưu vào Qdrant!
Dữ liệu đã được lưu vào Qdrant!
Dữ liệu đã được lưu vào Qdrant!
Dữ liệu đã được lưu vào Qdrant!


In [196]:
collection_info = qdrant_client.get_collection(collection_name=collection_name)
print(collection_info)

vectors = qdrant_client.scroll(
    collection_name="documents_collection",
    limit=10,
    with_payload=True,
)

vectors

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=7 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None)

([Record(id='04143984-d43b-4734-bad0-63ac4d3d22a8', payload={'file_path': 'C:\\Users\\LUNE\\AppData\\Local\\Temp\\tmphkfz7qvh\\TULD02.txt', 'file_name': 'TULD02.txt', 'file_type': 'text/plain', 'file_size': 16468, 'creation_date': '2024-12-02', 'last_modified_date': '2024-12-02', 'questions_this_excerpt_can_answer': 'Based on the provided context, here is a question that can be specifically answered:\n\n**Question:** What are the responsibilities of the employer (NSDLĐ) regarding employee safety training and health examinations as outlined in the document? \n\nThis question is specific to the content of the document and addresses the obligations of the employer concerning workplace safety and health, which may not be detailed in other sources.', 'excerpt_keywords': 'Keywords: employer responsibilities, employee safety training, health examinations, labor discipline, social insurance'}, vector=None, shard_key=None, order_value=None),
  Record(id='1a711557-273b-4f20-9bf2-9e61d27b4acb', p

In [197]:
all_points = []
scroll_token = None

while True:
    # Fetch points in batches
    response = qdrant_client.scroll(
        collection_name="documents_collection",
        with_vectors=True,  # Include vectors in the response
        with_payload=True,  # Include payloads in the response
        offset=scroll_token,  # Provide the scroll token for pagination
    )
    
    # Add retrieved points to the list
    all_points.extend(response[0])
    
    # Check if there's more data to fetch
    scroll_token = response[1]
    if scroll_token is None:  # No more data to fetch
        break
for point in all_points:
    print(f"ID: {point.id}, Vector: {point.vector}, Payload: {point.payload}")

ID: 04143984-d43b-4734-bad0-63ac4d3d22a8, Vector: [-0.00058760063, 0.03227993, 0.031167567, 0.004267625, 0.09403749, 0.037157215, -0.012706613, 0.0372, -0.013177228, -0.0037355088, 0.016760321, -0.0044146925, -0.027916044, -0.00626774, 0.012974008, -0.0016525014, -0.03189488, 0.03482553, 0.021241862, -0.0053773145, 0.01200069, 0.044580102, -0.005144681, 0.06028154, -0.00680253, 0.023274066, -0.026932029, -0.046334215, -0.0031793271, 0.0011531411, 0.036601033, -0.031595398, -0.0022995975, 0.034868315, 0.028643358, 0.05416354, -0.00055885565, 0.026247498, -0.013733409, -0.005909431, 0.023423806, -0.043660264, -0.033435076, -0.05027027, -0.019231051, 0.02033272, 0.01047119, 0.004021622, -0.012610351, 0.032451063, 0.015006211, 0.0080004595, -0.0031017826, 0.033841517, -0.0259908, 0.024193903, -0.0028798447, -0.03493249, -0.00080218515, 0.008963082, -0.020835422, -0.015016906, -0.014375158, 0.03441909, -0.008283898, -0.040472914, -0.05891248, -0.02626889, 0.007936285, -0.01974445, -0.012760

In [198]:
from llama_index.core import Document
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
# Chuyển đổi danh sách all_points thành danh sách Document
documents = [
    Document(
        id=point.id,  # ID của Document
        text=str(point.vector),  # Nội dung vector (hoặc chuyển vector thành chuỗi)
        metadata=point.payload  # Thêm metadata
    )
    for point in all_points
]

# Thêm vào docstore
docstore = SimpleDocumentStore()
docstore.add_documents(documents)
storage_context = StorageContext.from_defaults(
    docstore=docstore,
    vector_store=vector_store,
)

# retriver

In [199]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=1,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)
from llama_index.core.response.notebook_utils import display_source_node
retrieved_nodes = bm25_retriever.retrieve(
    "What do you know?"
)
for node in retrieved_nodes:
    print(node)

Node ID: 2dab9404-b690-4282-b940-d61df6927479
Text: [-0.020973377, 0.050685663, 0.043888737, -0.014885272,
0.05655044, 0.0028984044, -0.025634129, 0.05352095, 0.005277329,
-0.06637686, 0.013118071, 0.032489315, 0.03755788, -0.017701142,
0.00520936, 0.0034712881, -0.052511122, -0.03984942, 0.005092841,
0.014768753, -0.0050394367, 0.021051057, 0.015438736, 0.043966413,
-0.010341041, -0.021167576, -...
Score:  0.026



In [200]:
from llama_index.core.postprocessor import SentenceEmbeddingOptimizer
from llama_index.core.postprocessor import EmbeddingRecencyPostprocessor
from llama_index.core.postprocessor import LLMRerank


node_postprocessors = [
    SentenceEmbeddingOptimizer(
        embed_model=Settings.embed_model,
        # percentile_cutoff=0.5,
        threshold_cutoff=0.7,
    ),
    EmbeddingRecencyPostprocessor(date_key="date", similarity_cutoff=0.7),
    LLMRerank(top_n=2),
]

In [201]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers.type import ResponseMode

# Configure response synthesizer
response_synthesizer = get_response_synthesizer(llm=Settings.llm, response_mode=ResponseMode.COMPACT)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=bm25_retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

In [ ]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

CUSTOM_PROMPT = PromptTemplate(
    """
    Based on the conversation history between the User and the Assistant, along with the User's new question, analyze and understand the question within the context of the conversation.
    Provide a relevant response in Vietnamese, using a professional tone like a Human Resource Specialist.  

    <Conversation History>
    {chat_history}

    <Current Question>
    {question}
    """
)

custom_chat_history = [
    ChatMessage(
        role=MessageRole.USER,
        content="Hello assistant, we are having a conversation about the company's regulations.",
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="Great, would you like to know more information about the company's regulations?",
    ),
]

In [203]:
from llama_index.core.chat_engine import CondenseQuestionChatEngine

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=CUSTOM_PROMPT,
    chat_history=custom_chat_history,
    verbose=True
)

In [206]:
query = "Các hình thức xử phạt của công ty?"
# response = query_engine.query(query)
response = chat_engine.chat(query)

Querying with: Chào bạn,

Cảm ơn bạn đã đặt câu hỏi về các hình thức xử phạt của công ty. Theo quy định của công ty, các hình thức xử phạt có thể bao gồm:

1. **Nhắc nhở**: Đối với những vi phạm nhẹ, nhân viên sẽ nhận được nhắc nhở từ quản lý trực tiếp.
2. **Cảnh cáo**: Nếu vi phạm lặp lại hoặc nghiêm trọng hơn, nhân viên có thể bị cảnh cáo bằng văn bản.
3. **Giảm lương**: Trong một số trường hợp, công ty có thể áp dụng hình thức giảm lương tạm thời.
4. **Tạm đình chỉ công việc**: Đối với những vi phạm nghiêm trọng, nhân viên có thể bị tạm đình chỉ công việc trong một khoảng thời gian nhất định.
5. **Chấm dứt hợp đồng lao động**: Trong trường hợp vi phạm nghiêm trọng hoặc tái phạm nhiều lần, công ty có quyền chấm dứt hợp đồng lao động.

Mọi hình thức xử phạt đều được thực hiện theo quy trình rõ ràng và công bằng, đảm bảo quyền lợi cho nhân viên. Nếu bạn cần thêm thông tin chi tiết hoặc có thắc mắc cụ thể nào khác, xin vui lòng cho tôi biết.

Trân trọng.


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 11063 tokens (11063 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}